In [24]:
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.preprocessing import text,sequence
from keras import utils

print(tf.__version__)

2.12.0


In [25]:
df=pd.read_csv('Consumer_Complaints.csv')
df.head()

ParserError: ignored

In [9]:
col=['Consumer Complaint','Product']
df=df[col]
df=df[pd.notnull(df['Consumer Complaint'])]
df.head()

,Consumer Complaint,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection


import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_0, *['Consumer Complaint'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_1, *['Product'], **{})
chart

import numpy as np
from google.colab import autoviz

def heatmap(df, x_colname, y_colname, figscale=1, mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=(8 * figscale, 8 * figscale))
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(_df_2, *['Consumer Complaint', 'Product'], **{})
chart

In [26]:
df.isnull().sum()

Consumer Complaint    0
Product               0
dtype: int64

In [27]:
df['Product'].value_counts()

Debt collection                                                                 393
Mortgage                                                                        280
Credit reporting                                                                279
Credit card                                                                     197
Bank account or service                                                         136
Student loan                                                                     94
Consumer Loan                                                                    81
Payday loan                                                                      21
Money transfers                                                                  14
Prepaid card                                                                     11
Credit reporting, credit repair services, or other personal consumer reports      6
Other financial service                                                     

In [28]:
#split data

train_size=int(len(df)* .8)
print('train size- %d'%train_size)
print('test size- %d'%(len(df)-train_size))


train size- 1215
test size- 304


In [30]:
traincomplaint =df['Consumer Complaint'][:train_size]
trainproduct=df['Product'][:train_size]

testcomplaint= df['Consumer Complaint'][train_size:]
testproduct=df['Product'][train_size:]

In [31]:
max_words=1000
tokenize=text.Tokenizer(num_words=max,char_level=False)

tokenize.fit_on_texts(traincomplaint) #fit
xtrain=tokenize.texts_to_matrix(traincomplaint)
xtest=tokenize.texts_to_matrix(testcomplaint)


In [32]:
#sklearn to convert label strings to numbered index
encoder=LabelEncoder()
encoder.fit(trainproduct)
ytrain=encoder.transform(trainproduct)
ytest=encoder.transform(testproduct)

In [33]:
#convert labels  to one hot representation

num=np.max(ytrain)+1
ytrain=utils.to_categorical(ytrain,num)
ytest=utils.to_categorical(ytest,num)


In [34]:
print('xtrain shape',xtrain.shape)
print('xtest shape',xtest.shape)
print('ytrain shape',ytrain.shape)
print('ytest shape',ytest.shape)

xtrain shape (1215, 1000)
xtest shape (304, 1000)
ytrain shape (1215, 15)
ytest shape (304, 15)


In [38]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

max_words = 1000

#build model
model=Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [39]:
batch=32
epoch=5

history=model.fit(xtrain,ytrain,
                  batch_size=batch,
                  epochs=epoch,
                  verbose=1,
                  validation_split=0.1)

Epoch 1/5
35/35 [==============================] - 1s 9ms/step - loss: 1.9363 - accuracy: 0.3733 - val_loss: 1.6952 - val_accuracy: 0.5902
Epoch 2/5
35/35 [==============================] - 0s 5ms/step - loss: 1.0550 - accuracy: 0.7502 - val_loss: 1.3768 - val_accuracy: 0.6311
Epoch 3/5
35/35 [==============================] - 0s 5ms/step - loss: 0.6702 - accuracy: 0.8417 - val_loss: 1.2454 - val_accuracy: 0.6557
Epoch 4/5
35/35 [==============================] - 0s 5ms/step - loss: 0.4484 - accuracy: 0.9039 - val_loss: 1.2506 - val_accuracy: 0.6475
Epoch 5/5
35/35 [==============================] - 0s 5ms/step - loss: 0.3191 - accuracy: 0.9332 - val_loss: 1.2525 - val_accuracy: 0.6557


In [40]:
score=model.evaluate(xtest,ytest,batch_size=batch, verbose=1)

print(score)
print('test score-',score[0])
print('test accuracy',score[1])


10/10 [==============================] - 0s 2ms/step - loss: 0.7750 - accuracy: 0.7730
[0.7750194668769836, 0.7730262875556946]
test score- 0.7750194668769836
test accuracy 0.7730262875556946


In [47]:
test_label=encoder.classes_

for i in range(10):
  pred=model.predict(np.array([xtest[i]]))
  predlabel = test_label[np.argmax(pred)]
  print(testcomplaint.iloc[i][:50], "....")
  print('actual label- ' + testproduct.iloc[i])
  print('predicted - '+ predlabel + "\n")



1/1 [==============================] - 0s 24ms/step
I noticed on my latest credit report a collection  ....
actual label- Debt collection
predicted - Debt collection

1/1 [==============================] - 0s 21ms/step
I 'm filing a complaint because of the lack of hel ....
actual label- Mortgage
predicted - Mortgage

1/1 [==============================] - 0s 21ms/step
This is my second letter to the CFPB regarding the ....
actual label- Mortgage
predicted - Mortgage

1/1 [==============================] - 0s 22ms/step
I tried opening an Account with XXXX ... ..subject ....
actual label- Bank account or service
predicted - Bank account or service

1/1 [==============================] - 0s 21ms/step
I have lost my Discover card and copy of my social ....
actual label- Credit card
predicted - Credit card

1/1 [==============================] - 0s 22ms/step
In early XX/XX/2016, I tried logging into my Chase ....
actual label- Credit card
predicted - Credit card

1/1 [=====================